In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('DDosData/DDos_all.csv')

In [3]:
data.shape

(19639360, 81)

In [4]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'FlowDuration',
       'TotalFwdPackets', 'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s',
       'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCou

In [5]:
df = data.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [6]:
df.shape

(19639360, 78)

In [7]:
df['Label'].value_counts()

MSSQL      5787453
Syn        4284751
UDP        3754680
NetBIOS    3657497
LDAP       1915122
Portmap     186960
BENIGN       52897
Name: Label, dtype: int64

In [8]:
df = df.copy()
cat_columns = df.select_dtypes(['object']).columns
cat_columns

Index(['Label'], dtype='object')

In [9]:
df[cat_columns] = df[cat_columns].astype('category')
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df
data = df
data.dtypes

FlowDuration               float64
TotalFwdPackets            float64
TotalBackwardPackets       float64
TotalLengthofFwdPackets    float64
TotalLengthofBwdPackets    float64
                            ...   
IdleMean                   float64
IdleStd                    float64
IdleMax                    float64
IdleMin                    float64
Label                         int8
Length: 78, dtype: object

In [10]:
X = data.iloc[:, 0:-1].values
y = data.iloc[:, -1].values

In [11]:
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
X= standardScaler.fit_transform(X)
X

array([[-0.12880758,  0.00808122,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.15248456, -0.00550824,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.13130602,  0.00808122,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       ...,
       [-0.1524877 , -0.01230297,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.1524872 , -0.01230297,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.14469833,  0.00808122,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141]])

In [12]:
from keras.utils import np_utils
X=np.expand_dims(X,axis=2)
X.shape

(19639360, 77, 1)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [14]:
X_train.shape

(13158371, 77, 1)

In [15]:
X_test.shape

(6480989, 77, 1)

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

In [17]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(77,1)))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(8, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=1,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
394752/394752 [==============================] - 2596s 7ms/step - loss: 0.1126 - accuracy: 0.9717 - val_loss: 0.1147 - val_accuracy: 0.9732
2634.0436515808105


In [19]:
model.save('DDos_all_1epoch.h5')

In [20]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=1,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
394752/394752 [==============================] - 2521s 6ms/step - loss: 0.1155 - accuracy: 0.9736 - val_loss: 0.1011 - val_accuracy: 0.9750
2548.3205144405365


In [21]:
model.save('DDos_all_2epoch.h5')

In [22]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=1,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
394752/394752 [==============================] - 2539s 6ms/step - loss: 0.1176 - accuracy: 0.9738 - val_loss: 0.1032 - val_accuracy: 0.9753
2580.749171257019


In [23]:
model.save('DDos_all_3epoch.h5')